# 1. 使用langchain

In [ ]:
!pip install langchain

# 2. 导入第三方库

In [2]:
import os
import sys
import tarfile


from erniebot_agent.agents import RetrievalAgent
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import WholeMemory
from erniebot_agent.retrieval import BaizhongSearch
from erniebot_agent.tools.baizhong_tool import BaizhongSearchTool
from erniebot_agent.tools.langchain_retrieval_tool import LangChainRetrievalTool
from langchain.docstore.document import Document
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import SpacyTextSplitter
from erniebot_agent.utils.common import download_file

使用aistudio access token, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [3]:
aistudio_access_token = "<your aistudio access token>"
os.environ["EB_AGENT_ACCESS_TOKEN"] = aistudio_access_token
os.environ["AISTUDIO_ACCESS_TOKEN"] = aistudio_access_token

# 3. 预处理

## 3.1 下载数据集

In [4]:
if not os.path.exists("construction_regulations"):
    file_path = "construction_regulations.tar"
    download_file(
        url="https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar",
        save_path=file_path,
    )

    files = tarfile.open(file_path, "r:*")
    file_dir = os.path.dirname(file_path)
    files.extractall(file_dir, files.getmembers())
# ! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
# ! tar xvf construction_regulations.tar

## 3.2 构建知识库

In [5]:
embeddings = ErnieEmbeddings()

In [6]:
faiss_name = "faiss_index"
if os.path.exists(faiss_name):
    db = FAISS.load_local(faiss_name, embeddings)
else:
    loader = PyPDFDirectoryLoader("construction_regulations")
    documents = loader.load()
    text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=320, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_name)

# 4. 基于复杂Query分解的RetrievalAgent
## 4.1 复杂Query任务规划
在实际的问答场景，用户提出的Query往往是复杂的，比如用户问:"请比较一下波士顿和姚明出生的城市的区别是什么？"，这个问题涉及到了两个城市，并且还要比较两个城市的区别，如果直接把这个问题当成检索的Query，如果库里面没有直接回答这个问题的答案，那么这个问题就不能被正确检索，导致召回的结果与该Query的相关度偏低，最后就不能被正确的回复。

解决复杂Query的比较好的方法就是把复杂的Query分解成比较简单的子问题，比如上面的问题可以分解成：波士顿的人口，波士顿的GDP，姚明出生的城市，姚明出生城市的人口，姚明出生城市的GDP等。这些子问题一般都能够检索到正确的答案，最后大模型把这些子问题的答案进行汇总，得到这个query的最终的答案。

In [7]:
search_tool = LangChainRetrievalTool(db=db, threshold=0.0)
llm = ERNIEBot(model="ernie-3.5", api_type="aistudio")
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=False,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [8]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)

In [9]:
# 分解的子Query
for step in response.steps:
    print(step.info)

{'query': '城市设计管理办法的目的是什么？', 'name': 'sub query results 0'}
{'query': '城市设计管理办法的具体内容有哪些？', 'name': 'sub query results 1'}
{'query': '城市设计管理办法与城市规划有何关联？', 'name': 'sub query results 2'}
{'query': '城市设计管理办法如何实施？', 'name': 'sub query results 3'}
{'query': '城市设计管理办法有哪些特点或限制？', 'name': 'sub query results 4'}


In [10]:
# 子query检索结果，每个子query都搜索出了多篇文档，扩大了召回
for step in response.steps:
    print(step.result)

{'documents': [{'content': '住房和城乡建设部规章 \n   X住房和城乡建设部发布  \n- 1 -   城市设计管理办法 （2017年3月14日中华人民共和国住房和城乡建设部令第35号公布 自2017年6月1日起施行）    第一条 为提高城市建设水平，塑造城市风貌特色，推进城市设计工作，完善城市规划建设管理，依据《中华人民共和国城乡规划法》等法律法规，制定本办法。   第二条 城市、 县人民政府所在地建制镇开展城市设计管理工作，适用本办法。   第三条 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。', 'score': 0.36258640457409663, 'meta': {'source': 'construction_regulations/城市设计管理办法.pdf', 'page': 0}}, {'content': '有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。   第七条 城市设计分为总体城市设计和重点地区城市设计。\n\n  \n\n第八条 总体城市设计应当确定城市风貌特色， 保护自然山水格局， 优化城市形态格局， 明确公共空间体系， 并可与城市 （县人民政府所在地建制镇）总体规划一并报批。', 'score': 0.33495206470465577, 'meta': {'source': 'construction_regulations/城市设计管理办法.pdf', 'page': 1}}, {'content': '设计方案招标， 是指主要通过对投标人提交的设计方案进行评审确定中标人。', 'score': 0.321702275644589, 'meta': {'source': 'construction_regulations/建筑工程设计招标投标管理办法.pdf', 'page': 1}}]}
{'documents': [{'content': '有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。   第七条 城市设计分为总体城市设计和重点地区城市设计。\n\n  \n\n第八条 总体城市设计应当确定城市风貌特色， 保护自然山水格局， 优化城市形态格局， 明确公共空间体

In [11]:
# 运行的结果
response.text

'城市设计管理办法的具体内容包括：\n\n1. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n2. 城市设计分为总体城市设计和重点地区城市设计。总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。\n4. 设计方案招标，是指主要通过对投标人提交的设计方案进行评审确定中标人。'

## 4.2 使用compressor压缩子query检索结果

一个复杂的Query，往往需要分解成若干个子query，但是每个子query检索到的结果往往包含了很多不相关或者重复的信息，但这些信息会占用大量的Token，可能会导致大模型的token长度过长的情况。其中一个解决方法就是把每个子query检索到的信息进行压缩，即只提取与子query相关的片段，或者进行简短的总结即可，然后把总结或者提取的内容当成是子query的结果，这样会比直接使用子query的结果更好。

In [12]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [13]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)

In [14]:
# 分解的子Query
for step in response.steps:
    print(step.info)

{'query': '城市设计管理办法的目标是什么？', 'name': 'sub query compressor 0'}
{'query': '城市设计管理办法规定了哪些方面的要求？', 'name': 'sub query compressor 1'}
{'query': '城市设计管理办法中对于城市设计的原则有哪些规定？', 'name': 'sub query compressor 2'}
{'query': '城市设计管理办法中对于城市设计的实施过程有哪些规定？', 'name': 'sub query compressor 3'}
{'query': '城市设计管理办法中对于城市设计的监督和评估有哪些规定？', 'name': 'sub query compressor 4'}


In [15]:
# 压缩的子query检索结果，每个子query只提取了一篇文档，Token数被压缩
for step in response.steps:
    print(step.result)

{'content': '城市设计管理办法的目标是：\n1. 提高城市建设水平\n2. 塑造城市风貌特色\n3. 推进城市设计工作\n4. 完善城市规划建设管理', 'score': 0.3461942307827547, 'meta': {'source': 'construction_regulations/城市设计管理办法.pdf', 'page': 0}, 'sub_query': '城市设计管理办法的目标是什么？'}
{'content': '1. 第七条 城市设计分为总体城市设计和重点地区城市设计。\n2. 第八条 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 第十一条 历史文化街区和历史风貌保护相关控制地区开展城市设计，应当根据相关保护规划和要求，整体安排空间格局，保护延续历史文化，明确新建建筑和改扩建建筑的控制要求。\n4. 第十二条 城市设计重点地区范围以外地区，可以根据当地实际条件，依据总体城市设计，单独或者结合控制性详细规划等开展城市设计，明确建筑特色、公共空间和景观风貌等方面的要求。\n5. 第十四条 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。', 'score': 0.43130254209256125, 'meta': {'source': 'construction_regulations/城市设计管理办法.pdf', 'page': 1}, 'sub_query': '城市设计管理办法规定了哪些方面的要求？'}
{'content': '城市设计管理办法中对于城市设计的原则规定如下：\n\n1. 城市设计应当符合城市（县人民政府所在地建制镇）总体规划和相关标准。\n2. 城市设计应尊重城市发展规律，坚持以人为本，保护自然环境，传承历史文化，塑造城市特色，优化城市形态。\n3. 在开展城市设计时，组织编制机关应通过多种形式及渠道，广泛征求专家和公众意见。\n4. 审批前应依法进行公示，公示时间不少于30日。\n5. 城市设计成果应当自批准之日起20个工作日内，通过政府信息网站以及当地主要新闻媒体予以公布。\n6. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划

In [16]:
# 运行的结果
response.text

'城市设计管理办法的具体内容主要包括以下几个方面：\n\n1. 目标：提高城市建设水平，塑造城市风貌特色，推进城市设计工作，完善城市规划建设管理。\n2. 城市设计分类：总体城市设计和重点地区城市设计。总体城市设计应确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，可与城市（县人民政府所在地建制镇）总体规划一并报批。重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n3. 原则：城市设计应当符合城市（县人民政府所在地建制镇）总体规划和相关标准；应尊重城市发展规律，坚持以人为本，保护自然环境，传承历史文化，塑造城市特色，优化城市形态；在开展城市设计时，组织编制机关应通过多种形式及渠道，广泛征求专家和公众意见；审批前应依法进行公示，公示时间不少于30日；城市设计成果应当自批准之日起20个工作日内，通过政府信息网站以及当地主要新闻媒体予以公布。\n4. 实施过程：有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台；编制城市设计时，组织编制机关应当通过座谈、论证、网络等多种形式及渠道，广泛征求专家和公众意见；重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中；住房和城乡建设部、城市、县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况；城市、县人民政府城乡规划主管部门开展城市规划实施评估时，应当同时评估城市设计工作实施情况。\n5. 其他规定：以出让方式提供国有土地使用权，以及在城市、县人民政府所在地建制镇规划区内的大型公共建筑项目，应当将城市设计要求纳入规划条件；县级以上地方人民政府住房城乡建设主管部门依法对本行政区域内建筑工程设计招标投标活动实施监督，依法查处招标投标活动中的违法违规行为。\n\n以上内容仅供参考，如果需要更具体的信息可以查看《城市设计管理办法》原文件第四章“实施与监督”部分。'

## 4.3 使用few shot example 来引导子query分解

在利用大模型对复杂的Query分解成子query的时候，由于缺乏一些策略指导，大模型分解的子query会很随机，比如“请比较一下波士顿和姚明出生的城市的区别是什么？”，可能会被分解成：波士顿和姚明出生城市的人口，波士顿和姚明出生城市的GDP等。这些子Query虽然也比较简单具体，但不是最优的，不是我们预想的把两个城市拆开来生成子问题。因此，我们可以加入一些few shot样本来引导大模型按照我们的预想的方式来生成子问题，这样能够避免大模型出现很随机的子问题分解的情况。

In [31]:
# 根据GPT3的实验，few shot example 为3个比较合适，需要比较典型
data_map = {
    "电动汽车的品牌有哪些？各有什么特点？": [
        "当前市场上的主要电动汽车品牌。",
        "每个品牌的电动汽车品牌的基本技术规格，如续航里程、充电速度等。",
        "每个品牌的电动汽车型号，包括轿车、SUV、卡车等。",
        "每个品牌的充电基础设施和网络覆盖情况。",
        "每个品牌的电池技术和电动驱动系统。",
    ],
    "智能家居科技趋势及其在提高生活效率中的应用": ["当前最新的智能家居科技趋势是什么", "智能家居科技趋势的工作原理和特性", "哪些公司或产品正在引领这些智能家居科技趋势"],
    "请比较一下拼多多和淘宝的区别？": [
        "拼多多的产品定位是什么？",
        "淘宝的产品定位是什么？",
        "拼多多的GMV是多少？",
        "淘宝的GMB是多少？",
    ],
}

In [32]:
few_shot_examples = []
for key, values in data_map.items():
    meta_data = {}
    for i in range(len(values)):
        meta_data[f"sub_query_{i+1}"] = values[i]
    doc = Document(page_content=key, metadata={"sub_queries": meta_data})
    few_shot_examples.append(doc)

In [33]:
few_shot_retriever = FAISS.from_documents(few_shot_examples, embeddings)
few_shot_search = LangChainRetrievalTool(few_shot_retriever)

In [34]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    few_shot_retriever=few_shot_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [35]:
query = "请比较一下城市设计管理方法和城市照明管理规定的区别？"
response = await agent.run(query)

In [36]:
# 分解的子Query
for step in response.steps:
    if step.info["name"] != "few shot retriever":
        print(step.info)

{'query': '城市设计管理方法的定义是什么？', 'name': 'sub query compressor 0'}
{'query': '城市照明管理规定的定义是什么？', 'name': 'sub query compressor 1'}
{'query': '城市设计管理方法的目的是什么？', 'name': 'sub query compressor 2'}
{'query': '城市照明管理规定的目的是什么？', 'name': 'sub query compressor 3'}
{'query': '城市设计管理方法和城市照明管理规定的应用场景有何不同？', 'name': 'sub query compressor 4'}


In [37]:
# 运行的结果
response.text

'城市设计管理方法和城市照明管理规定在应用场景、目的、关注点和管理方式上存在一定的差异。\n\n1. 应用场景：城市设计管理方法主要关注城市整体规划和布局，应用于城市总体规划、区域规划、建筑设计、景观设计等方面。而城市照明管理规定更侧重于城市照明设施的设置和维护，以确保城市照明效果符合要求，并保证公共安全和市容市貌的美观。\n2. 目的和关注点：城市设计管理方法的目的是塑造城市风貌特色，完善城市规划建设管理。它关注整体平面和立体空间的统筹规划，建筑布局的协调，城市景观风貌的塑造，以及地域特征、民族特色和时代风貌的体现。而城市照明管理规定的目的是根据城市经济社会发展水平，结合城市自然地理环境、人文条件，按照城市总体规划确定的城市功能分区，对不同区域的照明效果提出要求。它更强调满足人们在户外活动和出行的安全需要，以及装饰和造景的作用。\n3. 管理方式：城市设计管理方法可能需要政府和相关部门的综合规划和协调。而城市照明管理规定则需要具体的技术标准和操作规范，以确保照明设施的安全、稳定和效果达标。\n\n在实际应用中，需要根据具体情况综合考虑，以实现更好的城市发展和管理效果。'

## 4.4 使用先验背景信息来引导子query分解
在利用大模型对复杂的Query分解成子query的时候，由于缺少先验知识，即不知道知识库里面包含了哪些内容，因此在做query分解的时候，会生成一些无用的子query，比如：“请比较一下波士顿和姚明出生的城市的区别是什么？”，如果库里面没有对美食的论述，而分解子query的时候出现了：波士顿有哪些美食，姚明的城市有哪些美食。这类无用的query，占了搜索资源，又对最终答案无效，会造成一定的浪费。因此，我们使用context retriever的方式，在做复杂query分解的时候，根据检索到的上下文内容进行分解，这样可以避免很多无效的子query分解的问题。

In [38]:
background_data = [
    "《城市管理执法办法》主要规定了城市管理执法的范围、主管部门、执法人员资格与行为准则、执法措施与程序，以及监督与责任追究机制，旨在确保城市管理执法活动规范、公正、文明，维护城市公共秩序和市容环境。",
    "《城市设计管理办法》主要内容是规定了城市设计的编制、审批、实施和管理的相关要求，旨在通过科学、合理的城市设计，塑造城市特色风貌，提升城市环境质量，促进城市可持续发展。该办法强调了城市设计的综合性、整体性和长远性，明确了各级政府和相关部门在城市设计中的职责和作用，为城市设计工作的规范化、制度化提供了保障。",
]

In [39]:
background_info = []
for item in background_data:
    meta_data = {}
    doc = Document(page_content=item)
    background_info.append(doc)

In [40]:
context_retriever = FAISS.from_documents(background_info, embeddings)
context_search = LangChainRetrievalTool(context_retriever)

In [41]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    context_retriever=context_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [42]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)

In [43]:
# 分解的子Query
for step in response.steps:
    if step.info["name"] != "context retriever":
        print(step.info)

{'query': '城市设计管理办法主要规定的内容是什么？', 'name': 'sub query compressor 0'}
{'query': '城市设计管理办法的具体实施和管理的相关要求是什么？', 'name': 'sub query compressor 1'}
{'query': '城市设计管理办法如何强调城市设计的综合性、整体性和长远性？', 'name': 'sub query compressor 2'}
{'query': '城市设计管理办法中各级政府和相关部门的职责和作用是什么？', 'name': 'sub query compressor 3'}
{'query': '城市设计管理办法如何保障城市设计工作的规范化、制度化？', 'name': 'sub query compressor 4'}


In [44]:
# 运行的结果
response.text

'城市设计管理办法的具体内容包括：\n\n1. 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n2. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n3. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n4. 编制城市设计时，组织编制机关应当通过座谈、论证、网络等多种形式及渠道，广泛征求专家和公众意见。\n5. 审批前应依法进行公示，公示时间不少于30日。\n6. 城市设计成果应当自批准之日起20个工作日内，通过政府信息网站以及当地主要新闻媒体予以公布。\n7. 有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。\n8. 城市、县人民政府城乡规划主管部门负责本行政区域内城市设计的监督管理。\n9. 城市、县人民政府城乡规划主管部门，应当充分利用新技术开展城市设计工作。\n10. 国务院和省、自治区人民政府城乡规划主管部门应当定期对各地的城市设计工作和风貌管理情况进行检查。\n\n这些规定和要求都强调了城市设计的目标和原则，以及其在整个城市规划建设管理过程中的重要性，可以有效地保障城市设计工作的规范化、制度化。'